### **Step 0: Latent Dirichlet Allocation**
LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions.

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial.
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution.

### **Step 1: Load the dataset**
The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the abcnews-date-text.csv file.

In [1]:
import pandas as pd
data=pd.read_csv('https://raw.githubusercontent.com/udacity/NLP-Exercises/master/2.2-topic-modeling/abcnews-date-text.csv', error_bad_lines=False)
data_text=data[:300000][['headline_text']]
data_text['index']=data_text.index
documents = data_text

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
print(len(documents))
documents[:5]

300000


,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


### **Step 2: Data Preprocessing**
We will perform the following steps:

* **Tokenization:** Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All stopwords are removed.
* Words are lemmatized - words in third person are changed to first person and verbs in past and future tenses are changed into present.
*Words are stemmed - words are reduced to their root form.

In [3]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

**Lemmatizer Example**

In [5]:
print(WordNetLemmatizer().lemmatize('went',pos='v'))

go


**Stemmer Example**

In [6]:
stemmer=SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles=[stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [7]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
  return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

# Tokenize and lemmatize
def preprocess(text):
  result=[]
  for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
      result.append(lemmatize_stemming(token))
  return result

In [8]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(" "):
  words.append(word)

print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))


Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


Let's now preprocess all the news headlines we have.

In [9]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs =documents['headline_text'].map(preprocess)

In [10]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

### **Step 3.1: Bag of words on the dataset**

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass **processed_docs** to **gensim.corpora.Dictionary()** and call it **'dictionary'**.

In [11]:
dictionary=gensim.corpora.Dictionary(processed_docs)

In [12]:
'''
Checking dictionary created
'''
count=0
for k,v in dictionary.iteritems():
  print(k,v)
  count+=1
  if count>10:
    break
 

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


**Gensim filter_extremes**

*filter_extremes(no_below=5, no_above=0.5, keep_n=100000)*

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [13]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=100000)

**Gensim doc2bow**

doc2bow(document)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [14]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [15]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(71, 1), (107, 1), (462, 1), (3530, 1)]

In [16]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 71 ("bushfir") appears 1 time.
Word 107 ("help") appears 1 time.
Word 462 ("rain") appears 1 time.
Word 3530 ("dampen") appears 1 time.


### **Step 3.2: TF-IDF on our document set**
While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.

**TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.
In other words:

* TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document).
* IDF(w) = log_e(Total number of documents / Number of documents with term w in it).

**For example**

* Consider a document containing 100 words wherein the word 'tiger' appears 3 times.
* The term frequency (i.e., tf) for 'tiger' is then:

   * TF = (3 / 100) = 0.03.
*Now, assume we have 10 million documents and the word 'tiger' appears in 1000 of these. Then, the inverse document frequency (i.e., idf) is calculated as:

    * IDF = log(10,000,000 / 1,000) = 4.

* Thus, the Tf-idf weight is the product of these quantities:

    * TF-IDF = 0.03 * 4 = 0.12.



In [17]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models


tfidf = models.TfidfModel(bow_corpus)

In [18]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''

corpus_tfidf = tfidf[bow_corpus]

In [19]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


### ***Step 4.1: Running LDA using Bag of Words***
We are going for 10 topics in the document corpus.

**We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* num_topics is the number of requested latent topics to be extracted from the training corpus.
*id2word is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* workers is the number of extra processes to use for parallelization. Uses all available cores by default.
* alpha and eta are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is ```1/num_topics```)

   * Alpha is the per document topic distribution.

      * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
      * Low alpha: Every document has a mixture of very few topics
   * Eta is the per topic word distribution.

    * High eta: Each topic has a mixture of most words(topics appear similar to each other).
    * Low eta: Each topic has a mixture of few words.

* **passes** is the number of training passes through the corpus. For example, if the training corpus has 50,000 documents, chunksize is 10,000, passes is 2, then online training is done in 10 updates:

  * #1 documents 0-9,999
  * #2 documents 10,000-19,999
  * #3 documents 20,000-29,999
  * #4 documents 30,000-39,999
  * #5 documents 40,000-49,999
  * #6 documents 0-9,999
  * #7 documents 10,000-19,999
  * #8 documents 20,000-29,999
  * #9 documents 30,000-39,999
  * #10 documents 40,000-49,999

In [20]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''

lda_model = gensim.models.LdaModel(bow_corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 2
                                   )

In [21]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.028*"miss" + 0.025*"closer" + 0.024*"reject" + 0.019*"search" + 0.015*"rudd" + 0.014*"play" + 0.014*"chief" + 0.013*"threat" + 0.012*"claim" + 0.012*"west" 
Words: 0


Topic: 0.019*"fear" + 0.019*"school" + 0.018*"boost" + 0.018*"worker" + 0.015*"union" + 0.015*"price" + 0.013*"expect" + 0.011*"win" + 0.010*"high" + 0.010*"crew" 
Words: 1


Topic: 0.054*"govt" + 0.036*"council" + 0.032*"urg" + 0.028*"plan" + 0.023*"fund" + 0.020*"call" + 0.018*"labor" + 0.017*"opposit" + 0.015*"back" + 0.015*"consid" 
Words: 2


Topic: 0.038*"kill" + 0.027*"crash" + 0.019*"deni" + 0.017*"die" + 0.015*"england" + 0.013*"train" + 0.013*"dead" + 0.013*"bomb" + 0.012*"storm" + 0.012*"farm" 
Words: 3


Topic: 0.030*"group" + 0.019*"want" + 0.017*"green" + 0.016*"offer" + 0.016*"govt" + 0.014*"open" + 0.013*"timor" + 0.012*"health" + 0.012*"delay" + 0.011*"need" 
Words: 4


Topic: 0.082*"polic" + 0.035*"charg" + 0.028*"face" + 0.026*"death" + 0.023*"investig" + 0.021*"court" + 0.021*"attack" + 0.021

**Classification of the topics**

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0:
* 1:
* 2:
* 3:
* 4:
* 5:
* 6:
* 7:
* 8:
* 9:

### **Step 4.2 Running LDA using TF-IDF**

In [23]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf,
                                             num_topics = 10, 
                                             id2word = dictionary,                                    
                                             passes = 2)

In [24]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.022*"water" + 0.012*"govt" + 0.012*"plan" + 0.012*"council" + 0.009*"fund" + 0.008*"drought" + 0.008*"firefight" + 0.008*"urg" + 0.007*"communiti" + 0.007*"farm"


Topic: 1 Word: 0.009*"condit" + 0.007*"doctor" + 0.007*"baghdad" + 0.007*"soldier" + 0.007*"govt" + 0.007*"alic" + 0.006*"uranium" + 0.006*"coal" + 0.006*"shortag" + 0.006*"withdraw"


Topic: 2 Word: 0.021*"polic" + 0.020*"charg" + 0.016*"murder" + 0.014*"miss" + 0.014*"search" + 0.013*"hick" + 0.013*"court" + 0.013*"rudd" + 0.012*"investig" + 0.012*"face"


Topic: 3 Word: 0.012*"labor" + 0.010*"bushfir" + 0.010*"elect" + 0.010*"feder" + 0.010*"murray" + 0.010*"liber" + 0.009*"highway" + 0.008*"poll" + 0.008*"govt" + 0.007*"burn"


Topic: 4 Word: 0.028*"crash" + 0.020*"kill" + 0.013*"die" + 0.012*"road" + 0.011*"polic" + 0.010*"iraq" + 0.010*"fatal" + 0.010*"driver" + 0.009*"dead" + 0.009*"injur"


Topic: 5 Word: 0.042*"closer" + 0.011*"timor" + 0.010*"nuclear" + 0.009*"law" + 0.008*"solomon" + 0.008*"polici

### **Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model**

We will check to see where our test document would be classified.

In [25]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [26]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# Our test document is document number 4310


# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3647572696208954	 
Topic: 0.058*"water" + 0.032*"warn" + 0.020*"sydney" + 0.019*"drought" + 0.017*"farmer" + 0.013*"coast" + 0.012*"rain" + 0.011*"busi" + 0.011*"minist" + 0.011*"lose"

Score: 0.2517940402030945	 
Topic: 0.023*"accus" + 0.022*"forc" + 0.017*"australian" + 0.016*"case" + 0.016*"break" + 0.015*"rule" + 0.014*"govt" + 0.013*"appeal" + 0.013*"decis" + 0.013*"begin"

Score: 0.24343976378440857	 
Topic: 0.019*"fear" + 0.019*"school" + 0.018*"boost" + 0.018*"worker" + 0.015*"union" + 0.015*"price" + 0.013*"expect" + 0.011*"win" + 0.010*"high" + 0.010*"crew"

Score: 0.02000420168042183	 
Topic: 0.054*"govt" + 0.036*"council" + 0.032*"urg" + 0.028*"plan" + 0.023*"fund" + 0.020*"call" + 0.018*"labor" + 0.017*"opposit" + 0.015*"back" + 0.015*"consid"

Score: 0.020002173259854317	 
Topic: 0.020*"lead" + 0.018*"elect" + 0.017*"resid" + 0.015*"nation" + 0.015*"industri" + 0.014*"public" + 0.014*"push" + 0.013*"plan" + 0.013*"region" + 0.012*"urg"

Score: 0.0200013201683759

For 10,000 news headlines, there should be more than 10 topics. But this exercise, we will limit to 10 topics.

### **Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model**



In [27]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5130008459091187	 
Topic: 0.010*"south" + 0.009*"break" + 0.008*"west" + 0.007*"control" + 0.007*"light" + 0.007*"hill" + 0.007*"speed" + 0.006*"east" + 0.006*"energi" + 0.006*"case"

Score: 0.32697588205337524	 
Topic: 0.012*"labor" + 0.010*"bushfir" + 0.010*"elect" + 0.010*"feder" + 0.010*"murray" + 0.010*"liber" + 0.009*"highway" + 0.008*"poll" + 0.008*"govt" + 0.007*"burn"

Score: 0.020010139793157578	 
Topic: 0.022*"water" + 0.012*"govt" + 0.012*"plan" + 0.012*"council" + 0.009*"fund" + 0.008*"drought" + 0.008*"firefight" + 0.008*"urg" + 0.007*"communiti" + 0.007*"farm"

Score: 0.02000552974641323	 
Topic: 0.011*"fire" + 0.010*"storm" + 0.009*"rat" + 0.008*"clean" + 0.007*"doubt" + 0.007*"sailor" + 0.007*"steal" + 0.007*"rescu" + 0.007*"grant" + 0.007*"weather"

Score: 0.02000223845243454	 
Topic: 0.011*"crew" + 0.009*"sentenc" + 0.009*"grower" + 0.008*"teacher" + 0.008*"councillor" + 0.008*"job" + 0.007*"retir" + 0.007*"scientist" + 0.007*"oper" + 0.007*"sack"

Score: 0

### **Step 6: Testing model on unseen document**

In [31]:
unseen_doc = "US closes 5 military bases in Afghanistan as part of Taliban peace deal"
bow_vector = dictionary.doc2bow(preprocess(unseen_doc))
bow_vector

[(416, 1), (820, 1), (854, 1), (964, 1), (2493, 1), (3417, 1), (3547, 1)]

In [32]:
for index, score in sorted(lda_model[bow_vector], key=lambda x: x[1], reverse=True): # sort by 2nd element in the tuple
    print(f"{index}, {score}, {lda_model.print_topic(index, 5)}")

8, 0.2625000476837158, 0.027*"aust" + 0.020*"iraq" + 0.018*"howard" + 0.017*"blaze" + 0.017*"indigen"
6, 0.26250001788139343, 0.020*"lead" + 0.018*"elect" + 0.017*"resid" + 0.015*"nation" + 0.015*"industri"
1, 0.13749998807907104, 0.019*"fear" + 0.019*"school" + 0.018*"boost" + 0.018*"worker" + 0.015*"union"
9, 0.13749997317790985, 0.058*"water" + 0.032*"warn" + 0.020*"sydney" + 0.019*"drought" + 0.017*"farmer"
2, 0.13749994337558746, 0.054*"govt" + 0.036*"council" + 0.032*"urg" + 0.028*"plan" + 0.023*"fund"
0, 0.012500000186264515, 0.028*"miss" + 0.025*"closer" + 0.024*"reject" + 0.019*"search" + 0.015*"rudd"
3, 0.012500000186264515, 0.038*"kill" + 0.027*"crash" + 0.019*"deni" + 0.017*"die" + 0.015*"england"
4, 0.012500000186264515, 0.030*"group" + 0.019*"want" + 0.017*"green" + 0.016*"offer" + 0.016*"govt"
5, 0.012500000186264515, 0.082*"polic" + 0.035*"charg" + 0.028*"face" + 0.026*"death" + 0.023*"investig"
7, 0.012500000186264515, 0.023*"accus" + 0.022*"forc" + 0.017*"australian" 

Recall topic #8 is about these words, which is about voilence.

Topic: 0.038*"kill" + 0.027*"crash" + 0.019*"deni" + 0.017*"die" + 0.015*"england" + 0.013*"train" + 0.013*"dead" + 0.013*"bomb" + 0.012*"storm" + 0.012*"farm""

The prediction about this headline is topic #8, which is not too bad.